**NASA-Inspired Research Assistant & Retrocompatibility Validation**

Copyright 2025, Denis Rothman

**Goal:** This notebook serves a dual purpose. Its primary goal is to demonstrate our fully upgraded Chapter 7 Context Engine as a complete, functional application: the **NASA-Inspired Research Assistant**. We will also perform rigorous validation by running **backward compatibility tests** for the key workflows from Chapters 5 and 6. This process showcases not only the engine's new capabilities for verifiability and security but also its stability as an evolving, enterprise-grade system.

This notebook demonstrates the following key capabilities:

* **System Validation & Backward Compatibility:** Before testing the new features, we will run the key workflows from previous chapters to prove that our latest upgrades have not compromised existing functionality, demonstrating the stability of our modular architecture.

* **High-Fidelity RAG in Action:** See the upgraded `Researcher` agent retrieve information from our new knowledge base and programmatically generate citations for its claims.

* **Agent Defenses at Work:** Observe how the `helper_sanitize_input` function provides a silent but essential layer of security, protecting the engine from potentially tainted data.

* **Complex, Dynamic Planning:** Analyze the technical trace to deconstruct the sophisticated, multi-step plan the `Planner` autonomously creates to answer a multi-faceted research question.

* **Verifiable, End-to-End Output:** The final result will be a clear, synthesized answer followed by a list of the source documents used, proving the system is not just powerful, but trustworthy.

# I. Inititalization

## GitHub

In [11]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [12]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch7/utils.py")
download_private_github_file("commons/ch7/helpers.py")
download_private_github_file("commons/ch7/agents.py")
download_private_github_file("commons/ch7/registry.py")
download_private_github_file("commons/ch7/engine.py")

✅ Successfully downloaded 'commons/ch7/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch7/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch7/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch7/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch7/engine.py' to './engine.py'


## Installation and client setup

In [13]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [14]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [15]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [16]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [17]:
#@title  Chapter 7: CONTROL DECK: NASA Research Assistant

# 1. Define a research goal that requires verifiable, cited answers.
goal = "What are the primary scientific objectives of the Juno mission, and what makes its design unique? Please cite your sources."

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


Definition
Juno is an orbital mission to Jupiter designed to investigate the planet’s origin and evolution, atmospheric properties, internal structure, and magnetospheric environment.

Function/Operation
- Employs a pole-to-pole (polar) trajectory, the first for an outer planet, to enable direct observations of Jupiter’s polar magnetosphere.
- Flies beneath Jupiter’s hazardous radiation belts, permitting closer, less shielded measurements of the planet’s fields and atmosphere.
- Conducts measurements of water abundance; characterizes atmospheric composition, temperature, and cloud motions; and maps gravity and magnetic fields to probe the deep interior.

Key Findings/Impact
- Enables tests of competing formation theories by quantifying water abundance and constraining Jupiter’s core mass, informing its origin and evolutionary history.
- Advances understanding of Jupiter’s meteorology and chemistry through detailed atmospheric characterization.
- Reveals deep internal structure via gravity mapping and provides direct exploration of the polar magnetosphere through its unique polar orbit.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 126.44 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Explain the mechanism or '
                                  'findings clearly and concisely.", '
                                  '"style_guide": "Maintain an objective and '
                                  'formal tone. Use precise terminology. '
                                  'Prioritize factual accuracy and clarity '
                                  'over narrative flair.", "structure": '
                                  '["Definition", "Function/Operation", "Key '
                                  'Findings/Impact"], "instruction": "Organize '
                                  'the provided facts into the defined '
                                  'structure, adhering to the style_guide."}'},
    'planned_input': { 'intent_query': 'Concise, expert science Q&A blueprint: '
        

In [18]:
#@title Chapter 6 CONTROL DECK: Context Reduction Workflow

# 1. Define a large piece of text that would be expensive or too long
# to use as direct context for the Writer agent.
large_text_from_researcher = """
Juno is a NASA space probe orbiting the planet Jupiter. It was launched from Cape Canaveral Air Force Station on August 5, 2011, as part of the New Frontiers program. Juno entered a polar orbit of Jupiter on July 5, 2016, to begin a scientific investigation of the planet. After completing its primary mission, it received a mission extension. Juno's mission is to measure Jupiter's composition, gravitational field, magnetic field, and polar magnetosphere. It is also searching for clues about how the planet formed, including whether it has a rocky core, the amount of water present within the deep atmosphere, mass distribution, and its deep winds, which can reach speeds up to 618 kilometers per hour (384 mph). Juno is the second spacecraft to orbit Jupiter, after the nuclear-powered Galileo orbiter, which orbited from 1995 to 2003. Unlike all earlier spacecraft to the outer planets, Juno is powered by solar arrays, which are commonly used by satellites orbiting Earth and working in the inner Solar System, whereas radioisotope thermoelectric generators are commonly used for missions to the outer Solar System and beyond. For Juno, however, the three largest solar array wings ever deployed on a planetary probe play an integral role in stabilizing the spacecraft and generating power.
"""

# 2. Define a goal that requires both using the large text AND a creative step.
# This forces the Planner to recognize the need for summarization before writing.
goal = f"""First, summarize the following text about the Juno probe to extract only the key facts about its scientific mission and instruments. Then, using that summary, write a short, suspenseful scene for a children's story about the probe's dangerous arrival at Jupiter.

--- TEXT TO USE ---
{large_text_from_researcher}
"""

# 3. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 4. Call the execution function
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


The giant fills my window. Black and gold. Bands and void.

I drift closer. My wings are wide. Three great panels. The largest any planetary probe has flown. They catch thin light. They hum. They steady me. They feed me.

Below, the planet pulls. Hard. I feel its grip in the tone of my gyros. In the shiver of the frame. Gravity presses. I listen. I measure. I map its field, piece by piece.

Something deeper waits. A question with weight. Is there a rocky heart? I ask it again and again. The answer hides in the numbers. In the way mass spreads inside the dark.

Static crawls along my skin. The magnetism sings. Low. Cold. I hear it in the crackle of my lines. I measure the magnetic field. I trace it toward the poles. Up where the polar magnetosphere snarls like wind in wires. It isn’t wind. It sounds like it.

I taste the thin breath above the clouds. I measure what it’s made of. I search for water far below. Not in drops. In clues. In how the deep atmosphere bends my signal. In how it weighs on me. Abundance or scarcity. The data will tell.

I watch the shadow of a ring pass over my wings. It moves slow. My arrays whisper as they flex. They keep me straight. They keep me alive.

Far beneath, other winds turn. Not the storms I can see. Older winds. Deeper. I listen for them. I feel their touch in tiny changes. In the sway of numbers. In the drift of my path.

I am small here. I am steady. I keep measuring. Composition. Gravity. Magnetism. The polar edge where fields break and roar.

The giant does not speak. It doesn’t need to. Its answers rise in silence. And I catch them, one by one, before the dark takes them back.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 46.00 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': '- Measures Jupiter’s composition, gravitational '
                           'field, magnetic field, and polar magnetosphere.\n'
                           '- Probes formation clues: presence of a rocky '
                           'core, water abundance in the deep atmosphere, '
                           'overall mass distribution, and deep atmospheric '
                           'winds.\n'
                           '- Powered by three very large solar array '
                           'wings—the largest flown on a planetary probe.\n'
                           '- These solar arrays both generate electrical '
                           'power and help stabilize the spacecraft.'},
    'planned_input': { 'summary_objective': 'Extract only the key facts about '
                                            "Juno's sc

In [19]:
#@title Chapter 5 Writing a story
# Define the high-level goal
goal = "Write a short, suspenseful scene for a children's story about the Apollo 11 moon landing, highlighting the danger."
# Call the execution function from the cell above
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


The room hummed. Cold screen light. Dust hung like tiny moons.

I opened the folder. Two names blinked back. juno_mission_overview.txt. perseverance_rover_tools.txt. My mouse clicked. The sound was small. It felt loud.

Jupiter swirled in a header. Juno watched from far away. Storms. Stripes. Silence between. I scrolled. The fan whispered. The clock ticked.

Next file. Mars grit and gears. Perseverance on red ground. Tools. Arms. Careful steps in thin air. I read fast. My heartbeat wasn’t.

I searched for something else. For a word that wouldn’t come. Apollo. Nothing. I tried again. Eagle. Nothing. West Crater. No. The screen stayed calm. Too calm.

The gap felt big. Like a dark hallway behind me. No callouts. No alarms. Just Jupiter above. Just Mars below. The middle was missing. The space between was wide and quiet.

I leaned closer. My breath fogged the glass. Letters swam. Juno. Perseverance. The words were steady. My fingers weren’t. Tap. Tap. Tap. The keys answered like distant knocks.

A shadow cut the screen. Just a sleeve. Just me. It still made my shoulders rise. I looked back anyway. Empty chair. Humming vents. The tick kept time.

I whispered the names. Juno to Jupiter. Perseverance to Mars. They stayed. The others didn’t. The silence pressed like a glove.

Somewhere, storms rolled over Jupiter. Somewhere, wheels pressed into Martian dust. Here, the missing thing waited. It didn’t shout. It didn’t move. It just wasn’t there.

I opened the files again. I listened. For a hint. For a slip. For a single word that didn’t belong.

Nothing. Just the hum. Just the tick. Just shadows pooling at the edge of the screen.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 57.49 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                               